In [15]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
from datetime import datetime

class FoundryLCAGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Foundry LCA Data Entry")
        
        # Create main frame
        main_frame = ttk.Frame(root, padding="10")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # Input section
        ttk.Label(main_frame, text="Input Parameters", font=('Helvetica', 12, 'bold')).grid(row=0, column=0, columnspan=4, pady=10)
        
        # Variables for storing input values
        self.inputs = {
            'coke': {'value': tk.StringVar(value='1500'), 'unit': tk.StringVar(value='tons per year'),
                    'per_unit': tk.StringVar(value='1000'), 'per_unit_type': tk.StringVar(value='kg')},
            'induction': {'value': tk.StringVar(value='5700'), 'unit': tk.StringVar(value='kw/h'),
                         'hours': tk.StringVar(value='10'), 'days': tk.StringVar(value='260')},
            'holding': {'value': tk.StringVar(value='265'), 'unit': tk.StringVar(value='kw/h'),
                       'hours': tk.StringVar(value='24'), 'days': tk.StringVar(value='260')},
            'pig_iron': {'value': tk.StringVar(value='900'), 'unit': tk.StringVar(value='tons per year'),
                        'per_unit': tk.StringVar(value='1000'), 'per_unit_type': tk.StringVar(value='kg')},
            'scrap': {'value': tk.StringVar(value='17100'), 'unit': tk.StringVar(value='tons per year'),
                     'per_unit': tk.StringVar(value='1000'), 'per_unit_type': tk.StringVar(value='kg')},
            'output': {'value': tk.StringVar(value='13000'), 'unit': tk.StringVar(value='tons per year'),
                      'per_unit': tk.StringVar(value='1000'), 'per_unit_type': tk.StringVar(value='kg')}
        }
        
        # Create input fields
        row = 1
        self.create_input_row(main_frame, row, "Coke (Cupola Furnace):", 'coke')
        row += 1
        self.create_input_row(main_frame, row, "Induction Furnace:", 'induction', True)
        row += 1
        self.create_input_row(main_frame, row, "Holding Machine:", 'holding', True)
        row += 1
        self.create_input_row(main_frame, row, "Pig Iron:", 'pig_iron')
        row += 1
        self.create_input_row(main_frame, row, "Secondary Scrap:", 'scrap')
        
        # Output section
        ttk.Label(main_frame, text="Output Parameters", font=('Helvetica', 12, 'bold')).grid(row=row+1, column=0, columnspan=4, pady=10)
        row += 2
        self.create_input_row(main_frame, row, "Iron Output:", 'output')
        
        # Save button
        ttk.Button(main_frame, text="Save Data", command=self.save_data).grid(row=row+1, column=0, columnspan=4, pady=20)

    def create_input_row(self, parent, row, label_text, key, is_electrical=False):
        ttk.Label(parent, text=label_text).grid(row=row, column=0, padx=5, pady=5)
        entry = ttk.Entry(parent, textvariable=self.inputs[key]['value'])
        entry.grid(row=row, column=1, padx=5, pady=5)
        ttk.Label(parent, textvariable=self.inputs[key]['unit']).grid(row=row, column=2, padx=5, pady=5)
        
        if is_electrical:
            frame = ttk.Frame(parent)
            frame.grid(row=row, column=3, padx=5, pady=5)
            ttk.Entry(frame, textvariable=self.inputs[key]['hours'], width=8).pack(side=tk.LEFT, padx=2)
            ttk.Label(frame, text="hours/day").pack(side=tk.LEFT, padx=2)
            ttk.Entry(frame, textvariable=self.inputs[key]['days'], width=8).pack(side=tk.LEFT, padx=2)
            ttk.Label(frame, text="days").pack(side=tk.LEFT, padx=2)
        else:
            frame = ttk.Frame(parent)
            frame.grid(row=row, column=3, padx=5, pady=5)
            kg_label = ttk.Label(frame, text="")
            kg_label.pack(side=tk.LEFT, padx=2)
            
            def update_kg(*args):
                try:
                    tons = float(self.inputs[key]['value'].get())
                    kg = tons * 1000
                    kg_label.config(text=f"{kg:,.0f} kg")
                    self.inputs[key]['per_unit'].set(kg)
                except ValueError:
                    kg_label.config(text="0 kg")
                    
            self.inputs[key]['value'].trace_add('write', update_kg)
            update_kg()

    def save_data(self):
        # Save LCA data to CSV
        data = {
            'Parameter': [],
            'Value': [],
            'Unit': [],
            'Additional Info': []
        }
        
        for key, values in self.inputs.items():
            data['Parameter'].append(key)
            data['Value'].append(values['value'].get())
            data['Unit'].append(values['unit'].get())
            if 'hours' in values:
                data['Additional Info'].append(f"{values['hours'].get()} hours/day, {values['days'].get()} days")
            else:
                data['Additional Info'].append(f"{values['per_unit'].get()} {values['per_unit_type'].get()}")

        df = pd.DataFrame(data)
        df.to_csv('foundry_lca_data.csv', index=False)

        # Update SimaPro CSV
        input_csv = 'input.CSV'
        output_csv = 'updated_simapro_file.CSV'
        self.update_simapro_csv(input_csv, output_csv)

        messagebox.showinfo("Success", "Data saved successfully to foundry_lca_data.csv and SimaPro file updated.")

    def update_simapro_csv(self, input_csv, output_csv):
        with open(input_csv, 'r', encoding='latin1') as infile:
            lines = infile.readlines()

        # Extract GUI data
        coke = float(self.inputs['coke']['per_unit'].get())
        output = float(self.inputs['output']['per_unit'].get())
        induction = float(self.inputs['induction']['value'].get())
        induction_hours = float(self.inputs['induction']['hours'].get())
        induction_days = float(self.inputs['induction']['days'].get())
        holding = float(self.inputs['holding']['value'].get())
        holding_hours = float(self.inputs['holding']['hours'].get())
        pig_iron = float(self.inputs['pig_iron']['per_unit'].get())
        scrap = float(self.inputs['scrap']['per_unit'].get())

        
        original_number = lines[31].strip()  
        current_time = datetime.now().strftime("%H%M%S")
        new_number = f"{original_number[:-6]}{current_time}"  
        lines[31] = lines[31].replace(original_number, new_number)

        # Row updates
        lines[113] = lines[113].replace("0.115", f"{coke / output}")
        calculated_value = (induction * induction_hours + holding * holding_hours) * induction_days
        lines[116] = lines[116].replace("1.2672", f"{calculated_value / output}")
        lines[139] = lines[139].replace("0.0692", f"{pig_iron / output}")
        lines[141] = lines[141].replace("1.3154", f"{scrap / output}")

        # Save the updated CSV
        with open(output_csv, 'w', encoding='latin1') as outfile:
            outfile.writelines(lines)

def main():
    root = tk.Tk()
    app = FoundryLCAGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()
